<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Functions-and-Classes" data-toc-modified-id="Functions-and-Classes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Functions and Classes</a></span></li><li><span><a href="#System-dependent-Configuration" data-toc-modified-id="System-dependent-Configuration-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>System-dependent Configuration</a></span></li></ul></li><li><span><a href="#Collect-Data" data-toc-modified-id="Collect-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Collect Data</a></span><ul class="toc-item"><li><span><a href="#Setup-the-Data-Collection-Environment" data-toc-modified-id="Setup-the-Data-Collection-Environment-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Setup the Data Collection Environment</a></span></li><li><span><a href="#Collect-Facebook-Post-Data" data-toc-modified-id="Collect-Facebook-Post-Data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Collect Facebook Post Data</a></span></li></ul></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

# Introduction

This playbook has been developed by the University of Florida team. @ 2020
<p> This playbook can harvest most data around a Tiktok search.</p>

<p> <b>INPUT:</b> FB credentials and a search term. </p>

<p> <b>OUTPUT</b> is written under data/raw in the format of <b>TODO: TIKTOK_POST_{Scrape_DateTime}_{Search_term}.csv</b> </p>

# Setup


<p> The imports, function and class defintions, global variables, and system-dependent configuration are in this section. </p>

<p> The system dependent configuration should be carefully reviewed and configured for each system (e.g., Linux vs. Windows, or the path of an external program) since the playbook will most likely fail without proper configuration. </p>

## Imports

In [1]:
"""This cell imports necessary Python modules and performs initial configuration
"""

### Data manipulation libraries
# import json
import pandas as pd 
import csv

### Visualization and Interaction
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')

from IPython.display import set_matplotlib_formats, display, clear_output, HTML
set_matplotlib_formats('retina')

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import VBox, HBox, Button, HTML, Label

### Computation libraries 
import numpy as np
import re
import random
import json

### Graph analysis
# import networkx as nx
# import community

### System related
# import sys
# import warnings;
# warnings.filterwarnings('ignore')

import io
import platform
from pathlib import Path

# from joblib import Parallel, delayed

### Datetime libraries
from datetime import datetime
import time
from time import sleep
from pytz import timezone
from getpass import getpass
### NLP dependencies
# import spacy
# from spacy.tokenizer import Tokenizer
# nlp = spacy.load('en')
# tokenizer = Tokenizer(nlp.vocab)

# from langdetect import detect

### Scraping libraries
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

### Machine learning libraries
# from sklearn import datasets
# from sklearn import linear_model
# from sklearn.feature_selection import f_regression, mutual_info_regression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

### Logging
import logging 
logging.basicConfig(level=logging.INFO)

#import spacy
# nlp = spacy.load('en')

## Parameters

In [2]:
"""This cell defines global variables and parameters used throughout the playbook
"""

# Set this to True if you want to watch Selenium scrape pages
WATCH_SCRAPING = True

# Set this to True if you want to use incognito mode
USE_INCOGNITO = True

# The data is written 
RAW_DATA_DIRECTORY = Path("PATH WHERE YOU WANT RESULTS/tiktok_results/")

# Setup logging level
LOGGING_LEVEL = logging.INFO 
logging.basicConfig(level=LOGGING_LEVEL)


## Functions and Classes

In [3]:
"""This cell defines functions and classes used throughout the playbook
"""

class Bot:
    def __init__(self, username, password, url): 
        
        #can add headless option if preferred, this is not headless for debugging purposes
        
        self.url = url
        #initiate browser
        self.opts = Options()
        self.opts.add_argument("user-agent=Googlebot")
        
#********************change chromdriver executable_path********************
        self.browser = webdriver.Chrome(executable_path="PATH TO CHROME DRIVER/chromedriver",options=self.opts)
        
        
        self.browser.implicitly_wait(5)
        self.browser.get('https://www.tiktok.com/login/?redirect_url=https%3A%2F%2Fwww.tiktok.com%2Ftrending%2F%3Flang%3Dfr&lang=en&enter_method=top_bar')
        sleep(2)
        self.browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[2]/div[3]/div[2]').click()
        sleep(2)

        # switch window
        root = self.browser.window_handles[0]
        self.browser.switch_to_window(self.browser.window_handles[1]) # switch to popup facebook login window

        # login with facebook
        self.browser.find_element_by_xpath('//*[@id="email"]').send_keys(username)
        self.browser.find_element_by_xpath('//*[@id="pass"]').send_keys(password)
        self.browser.find_element_by_xpath('//*[@id="loginbutton"]').click()
        sleep(10)

        #switch to first window
        self.browser.switch_to_window(root) 

        sleep(2)
        self.browser.get("https://www.tiktok.com/trending/?lang=en&loginType=facebook")
        
        #===================url of tag===================
        self.browser.get(self.url)
        
        sleep(2)
        
    def getContent(self):
        
        #get links to all Tik Tok videos in the tag
        video_links = []
        
        #this finds all elements with an href. An href is a link to the video.
        ids = self.browser.find_elements_by_xpath("//*[@href]")

        #retreive the href attribute and store it
        for h in ids:
            link = h.get_attribute('href')
            if ("?lang=en" and "/video/" in link):
                video_links.append(link)
        
        all_comments = []
        
        
        for vid in video_links:
            #get each video and scrape all data on the page besides the video content itself
            sleep(2)
            self.browser.get(vid)
            sleep(2)

            #Extended loop clicks to expand replies in relevant comments

            #finds all clickable buttons of this class name then attempts to click to expand
            replies = self.browser.find_elements_by_class_name("jsx-2071094126")
            
            for reply in replies:
                try:
                    if "View replies" in reply.text:
                        reply.click()
                        sleep(0.5)
                        
                        try:
                            for get in range(4): #clicks view more comments four times
                                
                                #find elements that start have a class name that starts with 'more'
                                more_replies = self.browser.find_elements_by_css_selector("p[class^='more']")
                                
                                for more in more_replies:
                                    try:
                                        more.click()
                                    except:
                                        pass
                        except:
                            pass
                except:
                    pass

            #the first element that starts with 'jsx' has the entire content of that page
            #includes likes, comments, usernames, dates comments were posted, and any associated hashtags
            objects = self.browser.find_element_by_css_selector("div[class^='jsx']")
            comments = objects.text
            all_comments.append("\n**New Post**\n")
            all_comments.append(comments)
            
        return all_comments
        
    def saveData(self, all_comments):
#*********************Change Storage path if preferred********************
        # save the data
        name_of_the_file = "TIKTOK_TAG_" + datetime.now().strftime("%Y-%m-%dT%H-%M-%S") + ".json"
        with open(RAW_DATA_DIRECTORY + name_of_the_file, 'w') as out_f:
            json.dump(all_comments, out_f)
        self.browser.close()
        return

## System-dependent Configuration

In [4]:
"""This cell defines system-dependent configuration such as those different in Linux vs. Windows
"""

# Get the system information from the OS
PLATFORM_SYSTEM = platform.system()

# Darwin is macOS
if PLATFORM_SYSTEM == "Darwin":
    EXECUTABLE_PATH = Path("CHROME DRIVER PATH/chromedriver")
elif PLATFORM_SYSTEM == "Windows":
    EXECUTABLE_PATH = Path("../dependencies/chromedriver.exe")
else:
    logging.critical("Chromedriver not found or Chromedriver is outdated...")
    exit()

# Collect Data

## Setup the Data Collection Environment

## Collect Post Data Using Your Facebook Login.

In [ ]:
"""This cell retrieves page posts and comments, for a given page.

# Setup the UI
text = widgets.Text(description="", width=200)
button = widgets.Button(description="Retrieve")
fb_selection = HBox([Label(value="Enter the full URL of FB Post:"), text, button])
display(VBox([HTML("<h2 style='text-align: center;'> Scrape a Facebook Post </h2>"), fb_selection]))

out = widgets.Output()
display(out)

def on_button_clicked(b):
    with out:
        clear_output()
        initial_page = text.value
        # posts_page = "https://www.facebook.com/" + str(text.value) + "/posts_to_page?ref=page_internal"
        # print("Retrieving posts and comments from " + str(initial_page))
 
        try:
            driver.get(initial_page)
            time.sleep(3)
        except:
            logging.info("Error retrieving the page. Try again.")
            
        # Go to the end of the page and make sure the pop-up appears
        # driver.execute_script("window.scrollTo(0, 10000)") 
        # time.sleep(3)   
            
            
        # df_comments = pd.DataFrame.from_dict(comment_row)
        # file_name = "FB_VIDEOPOST_" + datetime.now().strftime("%Y-%m-%dT%H-%M-%S") + "_" + initial_page.split("?v=")[1] + ".csv"
        # df_comments.to_csv(RAW_DATA_DIRECTORY / file_name , index=False, na_rep='None', columns=COLUMNS)
        # print(df_comments)
        
button.on_click(on_button_clicked)
"""
#==========================================================================
def main():
    username = input("Username: ") 
    password = getpass("password: ")
    
    hashtag = input("Enter search term:")
    #url of the tag to scrape
    hashtag = "https://www.tiktok.com/tag/" + hashtag + "?lang=en&loginType=facebook"
    
    #initiate instance
    tiktokBot = Bot(username, password,hashtag)
    
    #get content from most popular 30 videos
    all_content = tiktokBot.getContent()
    
    #save content in json format
    tiktokBot.saveData(all_content)

    sleep(50)
#==========================================================================
    
if __name__ == "__main__":
    main()